This file contains my approach to solving the Rossmann Sales Stores challenge which can be found on Kaggle: https://www.kaggle.com/c/rossmann-store-sales

The approach is similar to solving any Data Science problem:

1 - Understand the problem

2 - Inspect the given data

3 - Pre-process the data - clean it, modify it, enhance it

4 - Work with different kinds of models to see which one is the best fit for the given problem

5 - Train the model using the processed data

6 - Test the validity of the data

In [1]:
#Import all the required libraries

import numpy as np
import pandas
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split


/home/prakarsh/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Read the training data
columnDtype = {"Store": "int", "DayOfWeek": "int", "Date": "str", "Customers": "int", "Open": "int",
         "Promo": "int", "StateHoliday": "str", "SchoolHoliday": "int", "Sales": "int"}

storeData = pandas.read_csv("store.csv")
trainingData = pandas.read_csv("train.csv", parse_dates=[2], dtype=columnDtype)
trainingData.shape

(1017209, 9)

Inspecting the data is useful as it can help clean the data and make valid assumptions. Below, I would be removing certain elements from the data as a result of my inspection.

In [3]:
#Drop the "Customers" column because it is not present in the test.csv file. Even though it seems like a useful 
#feature for Sales prediction, its absence from test.csv helps us come to the decision that it will not impact the
#results
trainingData = trainingData.drop(["Customers"], axis=1)

#Remove rows where there are no Sales
trainingData = trainingData[trainingData["Open"] != 0]
trainingData = trainingData[trainingData["Sales"] > 0]

#Merge the Store data with Training data
trainingData = trainingData.merge(storeData, on="Store")
trainingData.shape

(844338, 17)

In [4]:
#This function performs a lot of preprocessing by computing new features from existing data that might help us
#make Sales prediction
def enhanceTrainingData(data):
    data.fillna(0, inplace=True)
    #Since we only have rows with Sales > 0, so the Open column should be 1. Replace null by 1.
    data.loc[data.Open.isnull(), "Open"] = 1

    #Replace the values in StoreType, Assortment and StateHoliday with Integer values
    category = {"0": 0, "a": 1, "b": 2, "c": 3, "d": 4}
    data.StoreType.replace(category, inplace=True)
    data.Assortment.replace(category, inplace=True)
    data.StateHoliday.replace(category, inplace=True)

    #Append date information using the parsed Date column and remove the actual column
    data.DayOfWeek = data.Date.dt.dayofweek
    data["Year"] = data.Date.dt.year
    data["Month"] = data.Date.dt.month
    data["Day"] = data.Date.dt.day
    data["WeekOfYear"] = data.Date.dt.weekofyear
    data = data.drop(["Date"], axis=1)

    data.CompetitionDistance = np.log1p(data.CompetitionDistance)

    #Compute the number of months since the closest competitor opened
    data["CompetitionSince"] = 12 * (data.Year - data.CompetitionOpenSinceYear) + (data.Month - data.CompetitionOpenSinceMonth)
    data = data.drop(["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear"], axis=1)

    #Compute the number of months since Promo2. Assume 4 weeks = 1 month
    data["PromoSince"] = 12 * (data.Year - data.Promo2SinceYear) + (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data["PromoSince"] = data.PromoSince.apply(lambda x: x if x > 0 else 0)
    data.loc[data.Promo2SinceYear == 0, "PromoSince"] = 0
    data = data.drop(["Year", "Promo2SinceWeek", "Promo2SinceYear"], axis=1)

    #Compute whether the current day falls under a Promo Month
    monthDictionary = {1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
                    7: "Jul", 8: "Aug", 9: "Sept", 10: "Oct", 11: "Nov", 12: "Dec"}
    data["MonthString"] = data.Month.map(monthDictionary)
    data.loc[data.PromoInterval == 0, "PromoInterval"] = ""
    data["IsPromoMonth"] = 0
    for interval in data.PromoInterval.unique():
        if interval != "":
            for month in interval.split(","):
                data.loc[(data.MonthString == month) & (data.PromoInterval == interval), "IsPromoMonth"] = 1
    
    data = data.drop(["MonthString", "PromoInterval", "Open"], axis=1)

    return data

In [5]:
def computeAverageSales(data, average_sales):
    data["AverageSales"] = 0
    for i in range(1, 1116):
        mean = data[data.Store == i].Sales.mean()
        mean = np.log1p(mean)
        data.loc[data.Store == i, "AverageSales"] = mean
        average_sales.append(mean)
    return data

In [6]:
trainingData

,Store,DayOfWeek,Date,Sales,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
5,1,6,2015-07-25,4364,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
6,1,5,2015-07-24,3706,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
7,1,4,2015-07-23,3769,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
8,1,3,2015-07-22,3464,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
9,1,2,2015-07-21,3558,1,0,0,0,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [7]:
trainingData = enhanceTrainingData(trainingData)
trainingData

,Store,DayOfWeek,Sales,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Month,Day,WeekOfYear,CompetitionSince,PromoSince,IsPromoMonth
0,1,4,5263,1,0,1,3,1,7.147559,0,7,31,31,82.0,0.0,0
1,1,3,5020,1,0,1,3,1,7.147559,0,7,30,31,82.0,0.0,0
2,1,2,4782,1,0,1,3,1,7.147559,0,7,29,31,82.0,0.0,0
3,1,1,5011,1,0,1,3,1,7.147559,0,7,28,31,82.0,0.0,0
4,1,0,6102,1,0,1,3,1,7.147559,0,7,27,31,82.0,0.0,0
5,1,5,4364,0,0,0,3,1,7.147559,0,7,25,30,82.0,0.0,0
6,1,4,3706,0,0,0,3,1,7.147559,0,7,24,30,82.0,0.0,0
7,1,3,3769,0,0,0,3,1,7.147559,0,7,23,30,82.0,0.0,0
8,1,2,3464,0,0,0,3,1,7.147559,0,7,22,30,82.0,0.0,0
9,1,1,3558,0,0,0,3,1,7.147559,0,7,21,30,82.0,0.0,0


In [8]:
trainingData.shape

(844338, 16)

Read the data to be tested, process it similar to trainingData

In [9]:
testData = pandas.read_csv("test.csv", parse_dates=[3])
testData.fillna(1, inplace=True)
testData = testData.merge(storeData, on="Store")
testData = enhanceTrainingData(testData)
testData.shape

(41088, 16)

Compute the average Sales for each of the 1115 stores and append that to the training and test data

In [10]:
average_sales = []

trainingData = computeAverageSales(trainingData, average_sales)
testData["AverageSales"] = 0
# 1115 = number of stores
for i in range(1, 1116):
    testData.loc[testData.Store == i, "AverageSales"] = average_sales[i - 1]

In [11]:
trainingData

,Store,DayOfWeek,Sales,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Month,Day,WeekOfYear,CompetitionSince,PromoSince,IsPromoMonth,AverageSales
0,1,4,5263,1,0,1,3,1,7.147559,0,7,31,31,82.0,0.0,0,8.468023
1,1,3,5020,1,0,1,3,1,7.147559,0,7,30,31,82.0,0.0,0,8.468023
2,1,2,4782,1,0,1,3,1,7.147559,0,7,29,31,82.0,0.0,0,8.468023
3,1,1,5011,1,0,1,3,1,7.147559,0,7,28,31,82.0,0.0,0,8.468023
4,1,0,6102,1,0,1,3,1,7.147559,0,7,27,31,82.0,0.0,0,8.468023
5,1,5,4364,0,0,0,3,1,7.147559,0,7,25,30,82.0,0.0,0,8.468023
6,1,4,3706,0,0,0,3,1,7.147559,0,7,24,30,82.0,0.0,0,8.468023
7,1,3,3769,0,0,0,3,1,7.147559,0,7,23,30,82.0,0.0,0,8.468023
8,1,2,3464,0,0,0,3,1,7.147559,0,7,22,30,82.0,0.0,0,8.468023
9,1,1,3558,0,0,0,3,1,7.147559,0,7,21,30,82.0,0.0,0,8.468023


Split the training data into train set (for training) and validation set (for measuring accuracy or validation)

In [12]:
X_train, X_valid = train_test_split(trainingData, train_size=0.8, random_state=10)

#Y_train = np.log1p(X_train.Sales)
Y_train = X_train.Sales    #So that we can use Logistic Regression
X_train = X_train.drop(["Sales"], axis=1)

#Y_valid = np.log1p(X_valid.Sales)
Y_valid = X_valid.Sales
X_valid = X_valid.drop(["Sales"], axis=1)

print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)

(675470, 16)
(675470,)
(168868, 16)
(168868,)


Try different models and pick which one works the best for this problem.
I am commenting out LogisticRegression and RandomForestRegressor as my Laptop is not powerful enough.

In [13]:
lm = LinearRegression()
lm.fit(X_train, Y_train)
print("Accuracy :: ", lm.score(X_valid, Y_valid))

"""
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(X_train, Y_train)
print(rf.score(X_valid, Y_valid))

lg = LogisticRegression()
lg.fit(X_train, Y_train)
print(lg.score(X_valid, Y_valid))
"""

Accuracy ::  0.710693090111913


'\nrf = RandomForestRegressor(n_estimators = 1000, random_state = 42)\nrf.fit(X_train, Y_train)\nprint(rf.score(X_valid, Y_valid))\n\nlg = LogisticRegression()\nlg.fit(X_train, Y_train)\nprint(lg.score(X_valid, Y_valid))\n'

Save the model which works the best

In [14]:
pickle.dump(lm, open("lr_model.dat", "wb"))

Use the model to make sales predictions

In [15]:
X_test = testData.drop(["Id"], axis=1)
predictions = lm.predict(X_test)

In [16]:
predictions

array([5867.42125575, 6015.20191365, 6162.98257156, ..., 8285.96635054,
       5254.80835633, 5402.58901423])

The saved model can be loaded in some other file to make predictions as well.
In the end, we can save these predictions in a file to match the Kaggle submission format.

In [17]:
result = pandas.DataFrame({"Id": testData["Id"], "Sales": predictions})
result.to_csv("Submission.csv", index=False)